In [ ]:
from huggingface_hub import login
login("")

In [2]:
!nvidia-smi

Thu Apr 17 09:18:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:01:00.0 Off |                    0 |
| N/A   61C    P0            165W /  250W |   18385MiB /  40960MiB |    100%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2",  
    model_kwargs={"device": device},  
    encode_kwargs={"normalize_embeddings": True}
)


/tmp/ipykernel_2540595/3445707328.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [4]:
import json
from langchain_community.document_loaders import JSONLoader

recipes_file = "final_recipes_lakhs.json"
jq_schema = ".[] | {title, ingredients, directions, total_time}"

loader = JSONLoader(
    file_path=recipes_file,
    jq_schema=jq_schema,
    text_content=False,  
    metadata_func=lambda record, metadata: {
        "title": record.get("title"),
        "total_time": record.get("total_time")
    },
    json_lines=False
)

data = loader.load()

In [5]:
for doc in data:
    content = json.loads(doc.page_content)
    doc.page_content = f"""
    Recipe Title: {content['title']}
    Ingredients: {content['ingredients']}
    Directions: {content['directions']}
    Total Time: {content['total_time']}
    """.strip()

In [6]:
print(data[0])

page_content='Recipe Title: Linda's Special Potato Salad
    Ingredients: 5 lbs    red potatoes, with skins on ; 2 cups    mayonnaise (more or less depending on how you like it, Do NOT use fat free or low fat) ; 1 large    onion, finely diced ; 2 stalks    celery, finely diced  or 1   teaspoon    celery seed ; 3 carrots, finely diced ; 1 green pepper, finely diced ; 6 hard-boiled eggs, finely chopped ; 3 tablespoons   fresh finely chopped chives ; 20 green olives, finely chopped ; 1⁄4 cup    yellow mustard ; 2 tablespoons    cider vinegar ; 1 tablespoon    sugar ; 1⁄2 teaspoon    salt ; 1 teaspoon    black pepper
    Directions: Peel potatoes, and boil until just done, about 20 minutes. (Just until your fork can be inserted all the way through the potato. You don't want them mushy). Drain well and let cool, then dice or slice potatoes. (I like a small dice, about 1/2"x1/2"). The potatoes will cut nicely once chilled and won't fall apart. Mix other ingredients, and add to chilled potato

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # Reduced for recipe context
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", "! ", "? ", "; ", ", "],  # Better semantic breaks
    add_start_index=True
)
all_splits = text_splitter.split_documents(data)

print(f"Split recipes into {len(all_splits)} sub-documents.")

Split recipes into 1191026 sub-documents.


In [8]:
# from langchain_community.vectorstores import FAISS

# vector_store = FAISS.from_documents(
#     documents=all_splits,
#     embedding=embeddings
# )

In [9]:
from langchain_community.vectorstores import FAISS
# Save the vector_store to local storage
# vector_store.save_local("vector_store_data")

# Load the vector_store from local storage
loaded_vector_store = FAISS.load_local("vector_store_data", embeddings, allow_dangerous_deserialization=True)

In [10]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# BM25 retriever
bm25_retriever = BM25Retriever.from_documents(all_splits)
bm25_retriever.k = 3

# ensemble retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[
        loaded_vector_store.as_retriever(search_kwargs={"k": 3}),
        bm25_retriever
    ],
    weights=[0.7, 0.3]
)

def retrieve_recipe(query, top_k=3):
    return ensemble_retriever.get_relevant_documents(query)[:top_k]

In [11]:
query = "What are the recipes which can be cooked in exactly 30 minutes?"
retrieved_docs = retrieve_recipe(query, top_k=3)
print(retrieved_docs)


/tmp/ipykernel_2540595/1805776034.py:17: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return ensemble_retriever.get_relevant_documents(query)[:top_k]


[Document(id='45ba67d6-884a-4cd1-8939-62515d70bc81', metadata={'title': 'Baked Rib Eye Steaks With Mushrooms', 'total_time': '26 mins', 'start_index': 1084}, page_content='. I can have supper on the table in 30 minutes. This recipe gives me time to make a salad and potatoes or pasta.'), Document(id='8f0f5d18-82dd-4a25-b218-72ffce3353b3', metadata={'title': 'Aussie Bacon and Egg Pie', 'total_time': '65 mins', 'start_index': 1101}, page_content='. Reduce the temperature to moderate and then cook for another 25 to 30 minutes. Serves hot or cold.'), Document(id='20b70f32-bc87-40e3-b75a-0f4e8cf1f50a', metadata={'title': 'Turkey Chili Soup', 'total_time': '360 mins', 'start_index': 1629}, page_content='. Add onions and garlic, sauteing until onions are translucent. Add celery, carrot, and potatoes. Sweat over medium heat for 30 minutes, until the vegetables are soft, stirring occasionally. Add chili powder, chipotle, cumin, paprika, and cinnamon, cooking over low heat for 10 minutes and stir

In [10]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from langchain.prompts import PromptTemplate

# # 1. Load Llama-2 7B
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     device_map="auto",
#     torch_dtype=torch.float16
# )

# # 2. Create Prompt Function
# def create_prompt(cooking_time):
#     return f"""<|user|>
# You are an expert cook. Generate a recipe that meets the following criteria:

# Cooking Time: The total cooking time must be ≤ {cooking_time} minutes. 
# Cooking includes boiling, baking, heating, frying, sautéing, grilling, roasting, or steaming.
# It does not include preparation, refrigeration, or cooling time.

# Format: The generated recipe must strictly follow this structure:

# Recipe Title: [Clear & concise name]
# Ingredients: [List ingredients with exact quantities]
# Instructions: [Step-by-step directions]

# The recipe must not exceed {cooking_time} minutes under any circumstances. Be precise about the time.
# </s>
# <|assistant|>
# """

# # 3. Response Generation Function
# def generate_llama_response(cooking_time, retrieved_docs):
#     # Format context from retrieved documents
#     context = "\n".join([d.page_content for d in retrieved_docs])
    
#     # Create the prompt dynamically based on cooking time
#     prompt = create_prompt(cooking_time)
    
#     # Combine context and prompt
#     full_prompt = f"{context}\n\n{prompt}"
    
#     # Tokenize and generate response
#     inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
#     outputs = model.generate(
#         inputs.input_ids,
#         max_new_tokens=512,
#         temperature=0.7,
#         top_p=0.9
#     )
    
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # 4. Full Pipeline Execution
# query = "What are the recipes which can be cooked in exactly 120 minutes?"
# retrieved_docs = retrieve_recipe(query, top_k=3)  # Assuming retrieve_recipe is defined elsewhere
# response = generate_llama_response(cooking_time=120, retrieved_docs=retrieved_docs)

# print("Final Answer:", response.strip())


In [ ]:
import ollama
import pickle
import random
from rapidfuzz import fuzz
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Ensure Ollama has LLaMA 2 7B installed
OLLAMA_MODEL = "llama2"

# Unique categories tracking
used_proteins = set()
used_cuisines = set()
used_methods = set()

# Available categories
all_proteins = {"chicken", "beef", "tofu", "lentils", "pork", "salmon", "mushrooms", "duck"}
all_cuisines = {"Italian", "Thai", "Korean", "Indian", "Mexican", "Moroccan", "French", "Chinese"}
all_methods = {"grilling", "stir-frying", "baking", "boiling", "steaming", "roasting"}

# Function to ensure variety
def get_unique_category(existing, all_options):
    available_options = list(all_options - existing)
    if not available_options:  # Reset if all are used
        existing.clear()
        available_options = list(all_options)
    choice = random.choice(available_options)
    existing.add(choice)
    return choice

# Generate recipe prompt with enforced uniqueness
def create_prompt(cooking_time):
    # unique_protein = get_unique_category(used_proteins, all_proteins)
    # unique_cuisine = get_unique_category(used_cuisines, all_cuisines)
    # unique_method = get_unique_category(used_methods, all_methods)

    return f"""<|user|>
                    You are an expert cook. Generate a recipe that meets the following criteria:


Cooking Time: The total cooking time must be <= {cooking_time} minutes. 
Cooking includes boiling, baking, heating, frying, sautÃ©ing, grilling, roasting, or steaming.
It does not include preparation, refrigeration, or cooling time.                    
Format: The generated recipe must strictly follow this structure:

                    Recipe Title: [Clear & concise name]
                    Ingredients: [List ingredients with exact quantities]
                    Instructions: [Step-by-step directions]


                        The recipe must not exceed {cooking_time} minutes under any circumstances. Be precise about the time.
                        </s>
                        <|assistant|>
                        """

# Retrieve diverse recipe samples
def retrieve_recipe(query, top_k=5):
    retrieved_docs = ensemble_retriever.get_relevant_documents(query)
    return random.sample(retrieved_docs, min(len(retrieved_docs), top_k))  # Random selection for variety

# Levenshtein similarity to prevent near-duplicates
def is_unique(recipe, existing_recipes, threshold=80):
    for r in existing_recipes:
        if fuzz.ratio(recipe, r) > threshold:
            return False
    return True

# Generate a recipe with enforced diversity
def generate_ollama_response(cooking_time, retrieved_docs):
    if not retrieved_docs:
        return "Error: No relevant recipes found."

    context = "\n".join([d.page_content for d in retrieved_docs])
    prompt = create_prompt(cooking_time)
    full_prompt = f"{context}\n\n{prompt}"

    try:
        response = ollama.chat(
            model=OLLAMA_MODEL,
            messages=[{"role": "user", "content": full_prompt}],
            options={"temperature": 0.7, "top_p": 0.5}  # Adjusted for creative diversity
        )
        return response["message"]["content"].strip()
    except Exception as e:
        return f"Error: Ollama execution failed. {e}"

# Generate 5 Unique Recipes per Cooking Time
times = [2, 5, 10, 20, 30, 40, 50, 60, 90, 120, 150, 180]
recipes_dict = {}

for idx, time in enumerate(times, start=1):
    print(f"Processing {idx}/{len(times)}: Generating recipes for {time} minutes...")

    unique_recipes = set()
    retrieved_docs = retrieve_recipe(f"Quick meals under {time} minutes", top_k=5)

    while len(unique_recipes) < 5:
        recipe = generate_ollama_response(time, retrieved_docs)
        if is_unique(recipe, unique_recipes):  # Ensure uniqueness
            unique_recipes.add(recipe)

    recipes_dict[time] = list(unique_recipes)

# Save Recipes to Pickle File
with open("unique_recipes_old.pkl", "wb") as f:
    pickle.dump(recipes_dict, f)

print("Recipes successfully generated and saved!")


In [11]:
import pickle

# Load the pickle file
with open("unique_recipes_6.pkl", "rb") as file:
    recipes_dict = pickle.load(file)

# Display the first 10 rows
for time, recipes in list(recipes_dict.items())[:10]:
    print(f"Cooking Time: {time} minutes")
    for recipe in recipes:
        print(recipe)
    print("-" * 50)

Cooking Time: 2 minutes
Title: Quick and Easy Posole in 2 Minutes!

Ingredients:

* 1 lb boneless, skinless chicken breast or thighs, cut into bite-sized pieces (300g)
* 1 onion, diced (150g)
* 2 cloves garlic, minced (30g)
* 1 red bell pepper, diced (150g)
* 1 can black beans, drained and rinsed (190g)
* 1 can diced tomatoes (140g)
* 1 tsp ground cumin (5g)
* Salt and pepper, to taste
* 4 cups vegetable broth (1L)

Instructions:

1. In a pot or instant pot, combine chicken, onion, garlic, red bell pepper, black beans, diced tomatoes, cumin, salt, and pepper.
2. Cook over high heat, stirring occasionally, until the chicken is cooked through and the vegetables are tender, about 2 minutes.
3. Stir in vegetable broth and bring to a simmer. Reduce heat to low and let it cook for another minute.
4. Serve hot, garnished with chopped cilantro or scallions (if desired). Enjoy!
Title: Quick & Easy Posole in 2 Minutes!

Ingredients:

* 1 lb boneless, skinless chicken breast or thighs, cut into b

In [ ]:
import pickle
import json
from rapidfuzz import fuzz

# Load the pickle file containing the recipes
with open("unique_recipes.pkl", "rb") as file:
    recipes_dict = pickle.load(file)

# Flatten the recipes into a list for evaluation
recipes = []
for time, recipe_list in recipes_dict.items():
    for recipe in recipe_list:
        recipes.append({
            "time": time,
            "recipe": recipe
        })

# Parameters for evaluation
def evaluate_recipes_with_scores(recipes, threshold=80):
    unique_recipes = set()
    evaluation_results = []
    total_scores = {"uniqueness": 0, "completeness": 0}
    total_recipes = len(recipes)

    for entry in recipes:
        recipe = entry["recipe"]
        time = entry["time"]

        # Check for uniqueness
        is_unique = all(fuzz.ratio(recipe, r) < threshold for r in unique_recipes)
        if is_unique:
            unique_recipes.add(recipe)
        uniqueness_score = 1 if is_unique else 0

        # Check for completeness
        has_title = "Recipe Title:" in recipe
        has_ingredients = "Ingredients:" in recipe
        has_instructions = "Instructions:" in recipe
        completeness_score = 1 if has_title and has_ingredients and has_instructions else 0

        # Update total scores
        total_scores["uniqueness"] += uniqueness_score
        total_scores["completeness"] += completeness_score

        # Evaluate the recipe
        evaluation_results.append({
            "time": time,
            "recipe": recipe.strip(),
            "is_unique": is_unique,
            "has_title": has_title,
            "has_ingredients": has_ingredients,
            "has_instructions": has_instructions,
            "uniqueness_score": uniqueness_score,
            "completeness_score": completeness_score,
        })

    # Calculate average scores
    average_scores = {
        "uniqueness": total_scores["uniqueness"] / total_recipes,
        "completeness": total_scores["completeness"] / total_recipes,
    }

    return evaluation_results, average_scores

# Evaluate the recipes
results, average_scores = evaluate_recipes_with_scores(recipes)

# Save the evaluation results to a JSON file
with open("evaluation_results_with_scores.json", "w") as file:
    json.dump({"results": results, "average_scores": average_scores}, file, indent=4)

print("Evaluation completed. Results saved to 'evaluation_results_with_scores.json'.")
print("Average Scores:", average_scores)

Evaluation completed. Results saved to 'evaluation_results_with_scores.json'.
Average Scores: {'uniqueness': 1.0, 'completeness': 0.08333333333333333}


In [ ]:
# import torch
# import pickle
# from transformers import AutoTokenizer, AutoModelForCausalLM
# from langchain.retrievers import BM25Retriever, EnsembleRetriever
# from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings

# #  Force CPU Execution
# device = torch.device("cpu")

# # 1. Load LLaMA-2 7B on CPU
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float32,  
#     low_cpu_mem_usage=True
# ).to(device)


# # 3. Recipe Generation
# def create_prompt(cooking_time):
#     return f"""<|user|>
# You are an expert cook. Generate a recipe that meets the following criteria:

# Cooking Time: The total cooking time must be ≤ {cooking_time} minutes. 
# Cooking includes boiling, baking, heating, frying, sautéing, grilling, roasting, or steaming.
# It does not include preparation, refrigeration, or cooling time.

# Format: The generated recipe must strictly follow this structure:

# Recipe Title: [Clear & concise name]
# Ingredients: [List ingredients with exact quantities]
# Instructions: [Step-by-step directions]

# The recipe must not exceed {cooking_time} minutes under any circumstances. Be precise about the time.
# </s>
# <|assistant|>
# """

# # 4. Retrieve Relevant Recipes (CPU Only)
# def retrieve_recipe(query, top_k=3):
#     return ensemble_retriever.get_relevant_documents(query)[:top_k]

# # 5. Generate Unique Recipes
# def generate_llama_response(cooking_time, retrieved_docs):
#     if not retrieved_docs:
#         return "Error: No relevant recipes found."

#     context = "\n".join([d.page_content for d in retrieved_docs])
#     prompt = create_prompt(cooking_time)
#     full_prompt = f"{context}\n\n{prompt}"

#     inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    
#     # Ensure input tensor is valid
#     if inputs.input_ids.shape[1] == 0:
#         return "Error: Model input is empty."

#     try:
#         outputs = model.generate(
#             inputs.input_ids,
#             max_new_tokens=512,
#             temperature=0.7,
#             top_p=0.9
#         )
#         recipe = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         return recipe.strip()
#     except RuntimeError as e:
#         return f"Error: Model execution failed. {e}"

# # 6. Generate 5 Unique Recipes for Each Time and Save as Pickle
# times = [2, 5, 10, 20, 30, 40, 50, 60, 90, 120, 150, 180]
# recipes_dict = {}

# for time in times:
#     unique_recipes = set()
#     retrieved_docs = retrieve_recipe(f"Recipes for {time} minutes", top_k=3)

#     while len(unique_recipes) < 5:
#         recipe = generate_llama_response(time, retrieved_docs)
#         if recipe not in unique_recipes:
#             unique_recipes.add(recipe)

#     recipes_dict[time] = list(unique_recipes)

# # Save to Pickle File
# with open("unique_recipes.pkl", "wb") as f:
#     pickle.dump(recipes_dict, f)

# print("Recipes successfully generated and saved!")
